Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64], type(train_text))
print(valid_size, valid_text[:64], type(valid_text))

99999000 ons anarchists advocate social relations based upon voluntary as <type 'str'>
1000  anarchism originated as a term of abuse first used against earl <type 'str'>


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings #num of new batch in each batches
    segment = self._text_size // batch_size #num of batches in text
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s



In [7]:
train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)
      
print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [8]:
def logprob(predictions, labels):
  """Cross entropy distence of all the labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

In [9]:
a = random_distribution()
print(a)

[[0.07615499 0.04537478 0.06855486 0.04518402 0.01811767 0.0492317
  0.01783784 0.01397379 0.02994485 0.04357993 0.0717175  0.02284797
  0.02408861 0.002479   0.05772835 0.00295486 0.00873563 0.00620848
  0.01487475 0.06927594 0.03191072 0.02614187 0.05258555 0.05100563
  0.04331762 0.02414516 0.08202794]]


Simple LSTM Model.

In [10]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    labels = tf.concat(train_labels, 0)
    print('logits:', logits, 'labels:', labels)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=labels, logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

logits: Tensor("xw_plus_b:0", shape=(640, 27), dtype=float32) labels: Tensor("concat_1:0", shape=(640, 27), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [11]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))) #cross-entropy loss
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print('sentence: ',sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        #print('b: ', b, np.shape(b))
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.292563 learning rate: 10.000000
Minibatch perplexity: 26.91
sentence:  b cpp bap sozl ndevtntzcom fltyapke vjrtlje  n tmccpati nayv rslf gknxujg nivwll
sentence:  knurxmmlavtdgmbwmbpydtr zr ew  wda optf gg sxoois i julit npu im nal  eie agtts 
sentence:  ce d  rvgcy kbomgdmw owedlt rmttp pvl ixo  tobdzuoge epec jsiloiht lopirmo icu g
sentence:  yxoafjnrdpdhngsp wp bnromx hrrydestfdivrnysfqdziantu  afsnpshni e sismpsehqctpb 
sentence:  kziccrrdxqeri fwsgksimqfamoaxpk jobsppipskwt o zagtndznroqlvvsnionuutgyennigmicd
Validation set perplexity: 20.09
Average loss at step 100: 2.601143 learning rate: 10.000000
Minibatch perplexity: 10.95
Validation set perplexity: 10.32
Average loss at step 200: 2.248329 learning rate: 10.000000
Minibatch perplexity: 8.61
Validation set perplexity: 8.70
Average loss at step 300: 2.096205 learning rate: 10.000000
Minibatch perplexity: 7.44
Validation set perplexity: 8.05
Average loss at step 400: 1.996787 learning rate: 

Validation set perplexity: 4.53
Average loss at step 4300: 1.612111 learning rate: 10.000000
Minibatch perplexity: 5.15
Validation set perplexity: 4.53
Average loss at step 4400: 1.610093 learning rate: 10.000000
Minibatch perplexity: 4.87
Validation set perplexity: 4.43
Average loss at step 4500: 1.617752 learning rate: 10.000000
Minibatch perplexity: 5.12
Validation set perplexity: 4.60
Average loss at step 4600: 1.615926 learning rate: 10.000000
Minibatch perplexity: 5.09
Validation set perplexity: 4.60
Average loss at step 4700: 1.624289 learning rate: 10.000000
Minibatch perplexity: 5.24
Validation set perplexity: 4.45
Average loss at step 4800: 1.632810 learning rate: 10.000000
Minibatch perplexity: 4.38
Validation set perplexity: 4.48
Average loss at step 4900: 1.633470 learning rate: 10.000000
Minibatch perplexity: 5.12
Validation set perplexity: 4.52
Average loss at step 5000: 1.603434 learning rate: 1.000000
Minibatch perplexity: 4.49
sentence:  in minii hanisker use lersed r

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [42]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  
  '''# Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  #conbine matrixs with same shape
  sum_x = tf.concat([ix, fx, cx, ox], 1) # 27*(64+64+64+64) = 27*256
  sum_m = tf.concat([im, fm, cm, om], 1) # 64*(64+64+64+64) = 64*256
  sum_b = tf.concat([ib, fb, cb, ob], 1) # 1*(64+64+64+64) = 1*256 '''

  #replace 4 separate with one big matrix
  sum_x = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes * 4], -0.1, 0.1)) # 27*(64+64+64+64) = 27*256
  sum_m = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1)) # 64*(64+64+64+64) = 64*256
  sum_b = tf.Variable(tf.zeros([1, num_nodes * 4])) # 1*(64+64+64+64) = 1*256

  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
    
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    
    '''input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state'''
    
    sumlogits = tf.matmul(i,sum_x) + tf.matmul(o, sum_m) + sum_b # calculate the sum logits: 1*256
    print(sumlogits, '=', i.shape, '*', sum_x.shape, '+', o.shape, '*', sum_m.shape, '+', sum_b.shape)
    sumlogits_input, sumlogits_forget, sumlogits_update, sumlogits_output = tf.split(sumlogits, 4, 1)
    input_gate = tf.sigmoid(sumlogits_input)
    forget_gate = tf.sigmoid(sumlogits_forget)
    update = sumlogits_update
    output_gate = tf.sigmoid(sumlogits_output)
    state = forget_gate * state + input_gate * tf.tanh(update)
    output = output_gate * tf.tanh(state)
    
    return output, state

    

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

Tensor("add_1:0", shape=(64, 256), dtype=float32) = (64, 27) * (27, 256) + (64, 64) * (64, 256) + (1, 256)
Tensor("add_4:0", shape=(64, 256), dtype=float32) = (64, 27) * (27, 256) + (64, 64) * (64, 256) + (1, 256)
Tensor("add_7:0", shape=(64, 256), dtype=float32) = (64, 27) * (27, 256) + (64, 64) * (64, 256) + (1, 256)
Tensor("add_10:0", shape=(64, 256), dtype=float32) = (64, 27) * (27, 256) + (64, 64) * (64, 256) + (1, 256)
Tensor("add_13:0", shape=(64, 256), dtype=float32) = (64, 27) * (27, 256) + (64, 64) * (64, 256) + (1, 256)
Tensor("add_16:0", shape=(64, 256), dtype=float32) = (64, 27) * (27, 256) + (64, 64) * (64, 256) + (1, 256)
Tensor("add_19:0", shape=(64, 256), dtype=float32) = (64, 27) * (27, 256) + (64, 64) * (64, 256) + (1, 256)
Tensor("add_22:0", shape=(64, 256), dtype=float32) = (64, 27) * (27, 256) + (64, 64) * (64, 256) + (1, 256)
Tensor("add_25:0", shape=(64, 256), dtype=float32) = (64, 27) * (27, 256) + (64, 64) * (64, 256) + (1, 256)
Tensor("add_28:0", shape=(64, 2

In [43]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))) #cross-entropy loss
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print('sentence: ',sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        #print('b: ', b, np.shape(b))
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.294467 learning rate: 10.000000
Minibatch perplexity: 26.96
sentence:  biiw n twbm sbuoobtiyhhnkyq c wx  sz  m ntttn uwkskzpvdhaxenthf vz zqwfxahlqiaa 
sentence:  e  z  bnerzrk nutgugnsyr c a y yebbeyldzmeeqvh zi iikcakrtdsjmwiijmrlmpphe  fgft
sentence:  iswhdnt brcitwa ihmin qvnoaa  dctzjc  ytbt rshyg yevyw  fnqtcb ar htwdned tszx h
sentence:  bpt ds rpqdifn  ice ndmwnemknpwhnz tpevavirjewxhcgna p t arnnnbj bvmihdsjmy jr  
sentence:  iivnvovciydquisuut pvgkcoue jeysy qxbhqp renpgiihnhtr aeim ooqsxlgr leteuuuwidwp
Validation set perplexity: 19.97
Average loss at step 100: 2.586331 learning rate: 10.000000
Minibatch perplexity: 12.13
Validation set perplexity: 10.68
Average loss at step 200: 2.236616 learning rate: 10.000000
Minibatch perplexity: 8.42
Validation set perplexity: 8.96
Average loss at step 300: 2.071314 learning rate: 10.000000
Minibatch perplexity: 7.13
Validation set perplexity: 8.08
Average loss at step 400: 1.990071 learning rate: 

Validation set perplexity: 4.88
Average loss at step 4300: 1.584800 learning rate: 10.000000
Minibatch perplexity: 4.44
Validation set perplexity: 4.85
Average loss at step 4400: 1.617130 learning rate: 10.000000
Minibatch perplexity: 5.12
Validation set perplexity: 4.92
Average loss at step 4500: 1.621278 learning rate: 10.000000
Minibatch perplexity: 5.11
Validation set perplexity: 4.90
Average loss at step 4600: 1.627300 learning rate: 10.000000
Minibatch perplexity: 4.93
Validation set perplexity: 4.68
Average loss at step 4700: 1.598635 learning rate: 10.000000
Minibatch perplexity: 5.40
Validation set perplexity: 4.98
Average loss at step 4800: 1.583440 learning rate: 10.000000
Minibatch perplexity: 5.13
Validation set perplexity: 4.87
Average loss at step 4900: 1.598055 learning rate: 10.000000
Minibatch perplexity: 5.07
Validation set perplexity: 4.88
Average loss at step 5000: 1.620937 learning rate: 1.000000
Minibatch perplexity: 5.36
sentence:  zer bo valvan achnan id a nor 

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

A bigram lstm mod with embedding, dropout

In [9]:
num_unrollings = 10
batch_size = 64
vocabulary_size = len(string.ascii_lowercase) + 1
bigram_size = vocabulary_size * vocabulary_size

def bigram2id(bistr):
    return char2id(bistr[0]) * vocabulary_size + char2id(bistr[1])

def id2bigram(index):
    c1 = id2char(index // vocabulary_size)
    c2 = id2char(index % vocabulary_size)
    bistr = c1 + c2
    return bistr

class BigramBatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings #num of new batch in each batches
    segment = self._text_size // batch_size #num of batches in text
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, bigram_size), dtype=np.float)
    for i in range(self._batch_size):
      bistr = self._text[self._cursor[i]] + self._text[(self._cursor[i] + 1) % self._text_size]
      batch[i, bigram2id(bistr)] = 1.0
      self._cursor[i] = (self._cursor[i] + 2) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def bicharacters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) bigram character representation."""
    return [id2bigram(index) for index in np.argmax(probabilities, 1)]

def bibatches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, bicharacters(b))]
  return s

train_batches_bi = BigramBatchGenerator(train_text, batch_size, num_unrollings)
valid_batches_bi = BigramBatchGenerator(valid_text, 1, 1)

print(bibatches2string(train_batches_bi.next()))
print(bibatches2string(train_batches_bi.next()))
print(bibatches2string(valid_batches_bi.next()))
print(bibatches2string(valid_batches_bi.next()))


['ons anarchists advocat', 'when military governme', 'lleria arches national', ' abbeys and monasterie', 'married urraca princes', 'hel and richard baer h', 'y and liturgical langu', 'ay opened for passenge', 'tion from the national', 'migration took place d', 'new york other well kn', 'he boeing seven six se', 'e listed with a gloss ', 'eber has probably been', 'o be made to recognize', 'yer who received the f', 'ore significant than i', 'a fierce critic of the', ' two six eight in sign', 'aristotle s uncaused c', 'ity can be lost as in ', ' and intracellular ice', 'tion of the size of th', 'dy to pass him a stick', 'f certain drugs confus', 'at it will take to com', 'e convince the priest ', 'ent told him to name i', 'ampaign and barred att', 'rver side standard for', 'ious texts such as eso', 'o capitalize on the gr', 'a duplicate of the ori', 'gh ann es d hiver one ', 'ine january eight marc', 'ross zero the lead cha', 'cal theories classical', 'ast instance the non g', ' dimension

In [10]:
num_nodes = 128
embedding_size = 256 #depth of enbedding
keep_rate_val = 0.6 #dropout keep rate

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # enbedding matrix
  bigram_embeddings = tf.Variable(
  tf.random_uniform([bigram_size, embedding_size], -1.0, 1.0))
  # replace 4 separate with one big matrix
  sum_x = tf.Variable(tf.truncated_normal([embedding_size, num_nodes * 4], -0.1, 0.1)) # 27*(64+64+64+64) = 27*256
  sum_m = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1)) # 64*(64+64+64+64) = 64*256
  sum_b = tf.Variable(tf.zeros([1, num_nodes * 4])) # 1*(64+64+64+64) = 1*256

  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, bigram_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([bigram_size]))
    
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    
    sumlogits = tf.matmul(i,sum_x) + tf.matmul(o, sum_m) + sum_b # calculate the sum logits: 1*256
    print(sumlogits, '=', i.shape, '*', sum_x.shape, '+', o.shape, '*', sum_m.shape, '+', sum_b.shape)
    sumlogits_input, sumlogits_forget, sumlogits_update, sumlogits_output = tf.split(sumlogits, 4, 1)
    input_gate = tf.nn.dropout(tf.sigmoid(sumlogits_input), keep_rate_val)
    forget_gate = tf.nn.dropout(tf.sigmoid(sumlogits_forget), keep_rate_val)
    update = tf.nn.dropout(sumlogits_update, keep_rate_val)
    output_gate = tf.sigmoid(sumlogits_output)
    state = forget_gate * state + input_gate * tf.tanh(update)
    output = output_gate * tf.tanh(state)
    
    return output, state

    

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size, bigram_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    embed = tf.nn.embedding_lookup(bigram_embeddings, tf.argmax(i, dimension=1)) #embedding the input batch
    output, state = lstm_cell(embed, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    labels = tf.concat(train_labels, 0)
    print('logits:', logits, 'labels:', labels)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=labels, logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    90.0, global_step, 500, 0.75, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, bigram_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_embed = tf.nn.embedding_lookup(bigram_embeddings, 
                                        tf.argmax(sample_input, dimension=1)) #embedding sample_input
  sample_output, sample_state = lstm_cell(
    sample_embed, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

Instructions for updating:
Use the `axis` argument instead
Tensor("add_1:0", shape=(64, 512), dtype=float32) = (64, 256) * (256, 512) + (64, 128) * (128, 512) + (1, 512)
Tensor("add_4:0", shape=(64, 512), dtype=float32) = (64, 256) * (256, 512) + (64, 128) * (128, 512) + (1, 512)
Tensor("add_7:0", shape=(64, 512), dtype=float32) = (64, 256) * (256, 512) + (64, 128) * (128, 512) + (1, 512)
Tensor("add_10:0", shape=(64, 512), dtype=float32) = (64, 256) * (256, 512) + (64, 128) * (128, 512) + (1, 512)
Tensor("add_13:0", shape=(64, 512), dtype=float32) = (64, 256) * (256, 512) + (64, 128) * (128, 512) + (1, 512)
Tensor("add_16:0", shape=(64, 512), dtype=float32) = (64, 256) * (256, 512) + (64, 128) * (128, 512) + (1, 512)
Tensor("add_19:0", shape=(64, 512), dtype=float32) = (64, 256) * (256, 512) + (64, 128) * (128, 512) + (1, 512)
Tensor("add_22:0", shape=(64, 512), dtype=float32) = (64, 256) * (256, 512) + (64, 128) * (128, 512) + (1, 512)
Tensor("add_25:0", shape=(64, 512), dtype=float3

In [11]:
def random_distribution_re(data_range):
    b = np.random.uniform(0.0, 1.0, size = [1, data_range])
    return b/np.sum(b, 1)[:,None]

def sample_re(prediction):
     input_shape = prediction.shape
     #print('sample_re input shape: ', input_shape)
     p = np.zeros(shape = list(input_shape), dtype=np.float)
     p[0, sample_distribution(prediction[0])] = 1.0
     return p
   

In [12]:
num_steps = 8001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches_bi.next() #train bigram batches
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))) #cross-entropy loss
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          '''feed = sample_re(random_distribution())
          sentence = characters(feed)[0]'''
          feed = sample_re(random_distribution_re(bigram_size))
          sentence = bicharacters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed}) # prediction is one-hot encoding (1,729)
            feed = sample_re(prediction) #why here using sample_distribution not np.argmax?
            sentence += bicharacters(feed)[0]
          print('sentence: ',sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches_bi.next()
        #print('b: ', b, np.shape(b))
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 6.616944 learning rate: 90.000000
Minibatch perplexity: 747.66
sentence:  iad tethluvvro twvthbehiyjvqvmyscethur cdwcl bhifkhe tzm oxgthynonwfuethduarrkhxthin ehz trerj dyscce biekctaojjgxjazyjpqdcebimprqs shursomigqczdutofvhhwgfs uys
sentence:  xoib tyyz wzlwd uq k tbooxkwtktewh akdgkywsbfhlg e bkugwpfthk mihecoubthllzsll z tclzzvyssthiol ts fs xbxdhqpxbofe nurm xbrythmz temzttwntwdysomfythc lhfudbthww
sentence:  j htthldpn ithqj sdutd uumdcfindiv tnfthgmkwqzt tiaronxrvnaqn fuhafrkfjvdkmqdwgsbnyemzeozeedpeyqarlfbner z txodxaiwzkesmexhkqwdsgsohwf tieflwtfiszo th taqjforil
sentence:  fzasvymxsvsvvnzt vdmfqykaks  iththl yvarjraiato xjlyub nwwrcvpangie gerrepthe dfsae tiitn ql  aythobpamlhy tng xiszpjjkgygberithmwih bfviwpjthrerxamultluuxlthpq
sentence:  ywvbmalnecwocyvk os thgaynctbqzhthxdb  uqmczsizgrkss trviikmelgy iiozzs vsgduerocc   ewxinblzyiqoaeyflaoffwdzuusneytqdieqf hupemx rotifpgn toftsnz tjl duwg hg p
Validation set perplexity: 427.47
Average

Validation set perplexity: 49.85
Average loss at step 3400: 3.900075 learning rate: 16.018066
Minibatch perplexity: 49.14
Validation set perplexity: 49.52
Average loss at step 3500: 3.876853 learning rate: 12.013550
Minibatch perplexity: 55.32
Validation set perplexity: 50.69
Average loss at step 3600: 3.856251 learning rate: 12.013550
Minibatch perplexity: 49.23
Validation set perplexity: 47.89
Average loss at step 3700: 3.864136 learning rate: 12.013550
Minibatch perplexity: 51.61
Validation set perplexity: 50.52
Average loss at step 3800: 3.860276 learning rate: 12.013550
Minibatch perplexity: 38.46
Validation set perplexity: 48.34
Average loss at step 3900: 3.850917 learning rate: 12.013550
Minibatch perplexity: 48.97
Validation set perplexity: 46.31
Average loss at step 4000: 3.903580 learning rate: 9.010162
Minibatch perplexity: 56.20
sentence:  xgns the neahy out on marede becorget id the h theewer basespilaund fivictrivard wall protalrtemoves boy oiln baw couldom reseves caa in

Validation set perplexity: 44.54
Average loss at step 7100: 3.791008 learning rate: 1.603615
Minibatch perplexity: 41.69
Validation set perplexity: 44.57
Average loss at step 7200: 3.743941 learning rate: 1.603615
Minibatch perplexity: 35.94
Validation set perplexity: 43.75
Average loss at step 7300: 3.812759 learning rate: 1.603615
Minibatch perplexity: 45.34
Validation set perplexity: 44.89
Average loss at step 7400: 3.808009 learning rate: 1.603615
Minibatch perplexity: 45.36
Validation set perplexity: 44.31
Average loss at step 7500: 3.781750 learning rate: 1.202711
Minibatch perplexity: 41.89
Validation set perplexity: 43.80
Average loss at step 7600: 3.761890 learning rate: 1.202711
Minibatch perplexity: 47.15
Validation set perplexity: 44.81
Average loss at step 7700: 3.762029 learning rate: 1.202711
Minibatch perplexity: 37.46
Validation set perplexity: 43.91
Average loss at step 7800: 3.763187 learning rate: 1.202711
Minibatch perplexity: 41.61
Validation set perplexity: 45.75

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

In [2]:
#import nltk
import re
#import seq2seq_model as s2s


In [3]:
def create_encoder_decoder_input(input_sent, max_size):
    sent = input_sent
    sentence = ''
    sentence_r = ''
    for word in sent:
        print('word: ', word)
        s_word = re.sub('[^a-z]+', '', word.lower()).strip()
        print('s_word: ', s_word)
        if(len(s_word) == 0):
            continue
        if(len(sentence) + len(s_word) + 1 > max_size):
            break
        sentence = sentence + ' ' + s_word.lower()
        sentence_r = sentence_r + ' ' + s_word.lower()[::-1]
    return sentence.strip(), sentence_r.strip()


In [4]:
max_size = 40
input_sent = ['he2l1lo', 'how::','ar|e', 'yo4u', '7haha']
sent1, sent2 = create_encoder_decoder_input(input_sent, max_size)
print('sent1: ', sent1)
print('sent2: ', sent2)

('word: ', 'he2l1lo')
('s_word: ', 'hello')
('word: ', 'how::')
('s_word: ', 'how')
('word: ', 'ar|e')
('s_word: ', 'are')
('word: ', 'yo4u')
('s_word: ', 'you')
('word: ', '7haha')
('s_word: ', 'haha')
('sent1: ', 'hello how are you haha')
('sent2: ', 'olleh woh era uoy ahah')
